In [28]:
import pandas as pd
import keras
import numpy as np
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = 9,6

from keras.datasets import cifar10

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, UpSampling2D
from tensorflow.keras.layers import LeakyReLU

from keras import backend as K
from keras import applications
from keras import callbacks
from keras import optimizers
from tensorflow.keras.utils import to_categorical
import random
from sklearn.metrics import accuracy_score
import tensorflow as tf
import tensorflow_addons as tfa
""
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, LeakyReLU, MaxPooling2D, Dropout, Flatten, Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers.schedules import ExponentialDecay

import warnings
warnings.filterwarnings(action='ignore')

# Data loading and preparation

In [29]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [30]:
BATCH_SIZE = 32
INPUT_SHAPE = X_train.shape[1:]
NUM_CLASSES=10

print('x_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
50000 train samples
10000 test samples


In [31]:
# Normalize input data
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

# Convert class labels to one-hot encoded
y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = to_categorical(y_test, num_classes=NUM_CLASSES)

# Fitting the Deepl learning models

In [32]:
LR=1e-3 # chosen arbitrary, tuning performed later on for further experiments
EPOCHS=100  
PATIENCE=10
MIN_DELTA=0.01
early_stop = EarlyStopping(monitor='val_accuracy', patience=PATIENCE, min_delta=MIN_DELTA)

In [33]:
def generate_model1(input_shape, 
                    num_classes, 
                    learning_rate=0.001,
                    dropout_rate1=0.25,
                    dropout_rate2=0.5,
                    weight_decay=0.0001,
                    decay_steps=100000,
                    decay_rate=0.96):
    """
    Generate Keras Sequential model according to proposed architecture (1).

    Args:
        learning_rate (float, optional): Learning rate of the neural network. Defaults to LR.

    Returns:
        Keras Sequential model, complied, ready to use (to call .fit method).
    """

    model = Sequential([
        Conv2D(16, (3, 3), padding='same', kernel_regularizer=l2(weight_decay), input_shape=input_shape),
        LeakyReLU(0.1),
        Conv2D(32, (3, 3), padding='same', kernel_regularizer=l2(weight_decay)),
        LeakyReLU(0.1),
        MaxPooling2D((2, 2)),
        Dropout(dropout_rate1),
        Conv2D(32, (3, 3), padding='same', kernel_regularizer=l2(weight_decay)),
        LeakyReLU(0.1),
        Conv2D(64, (3, 3), padding='same', kernel_regularizer=l2(weight_decay)),
        LeakyReLU(0.1),
        MaxPooling2D((2, 2)),
        Dropout(dropout_rate1),
        Flatten(),
        Dense(256, kernel_regularizer=l2(weight_decay)),
        LeakyReLU(0.1),
        Dropout(dropout_rate2),
        Dense(num_classes, activation='softmax')
    ])

    lr_schedule = ExponentialDecay(
        initial_learning_rate=learning_rate,
        decay_steps=decay_steps,
        decay_rate=decay_rate,
        staircase=True)

    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    
    model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, 
              metrics=['accuracy'])
    return model

In [34]:
model1 = generate_model1(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)
model1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 32, 32, 16)        448       
                                                                 
 leaky_re_lu_15 (LeakyReLU)  (None, 32, 32, 16)        0         
                                                                 
 conv2d_13 (Conv2D)          (None, 32, 32, 32)        4640      
                                                                 
 leaky_re_lu_16 (LeakyReLU)  (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 16, 16, 32)        0         
 g2D)                                                            
                                                                 
 dropout_9 (Dropout)         (None, 16, 16, 32)        0         
                                                      

In [23]:
model2 = generate_model2()
model2.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_34 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 conv2d_35 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_17 (MaxPooli  (None, 16, 16, 32)        0         
 ng2D)                                                           
                                                                 
 dropout_25 (Dropout)        (None, 16, 16, 32)        0         
                                                                 
 conv2d_36 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 conv2d_37 (Conv2D)          (None, 16, 16, 64)        36928     
                                                      

In [24]:
def generate_model3(learning_rate: float = LR):
    """
    Generate Keras Sequential model according to proposed architecture (3).

    Args:
        learning_rate (float, optional): Learning rate of the neural network. Defaults to LR.

    Returns:
        Keras Sequential model, complied, ready to use (to call .fit method).
    """

    model = Sequential()

    model.add(Conv2D(filters=32, kernel_size=(4,4), input_shape=INPUT_SHAPE, activation='relu',))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=32, kernel_size=(4,4), activation='relu',))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))
    
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=learning_rate,
        decay_steps=100000,
        decay_rate=0.96,
        staircase=True)

    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    
    model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, 
              metrics=['accuracy'])
    
    return model

In [25]:
model3=generate_model3()
model3.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_40 (Conv2D)          (None, 29, 29, 32)        1568      
                                                                 
 max_pooling2d_20 (MaxPooli  (None, 14, 14, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_41 (Conv2D)          (None, 11, 11, 32)        16416     
                                                                 
 max_pooling2d_21 (MaxPooli  (None, 5, 5, 32)          0         
 ng2D)                                                           
                                                                 
 flatten_9 (Flatten)         (None, 800)               0         
                                                                 
 dense_18 (Dense)            (None, 128)              